In [1]:
!pip install -q transformers datasets scikit-learn accelerate

import numpy as np
import pandas as pd

from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    set_seed
)

from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from datasets import Dataset


In [2]:
ds = load_dataset("unswnlporg/BESSTIE")
ds

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train.csv: 0.00B [00:00, ?B/s]

valid.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/17760 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2428 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'variety', 'source', 'task'],
        num_rows: 17760
    })
    validation: Dataset({
        features: ['text', 'label', 'variety', 'source', 'task'],
        num_rows: 2428
    })
})

In [3]:
def filter_hf(dataset, task, variety=None, source=None):
    def fn(ex):
        if ex["task"] != task:
            return False
        if variety is not None and ex["variety"] != variety:
            return False
        if source is not None and ex["source"] != source:
            return False
        return True
    return dataset.filter(fn)

def has_both_labels(dataset):
    labels = set(dataset["label"])
    return (0 in labels) and (1 in labels)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)

    acc = accuracy_score(labels, preds)

    # importante: forziamo entrambe le label (0 e 1) e gestiamo divisione per zero
    f1 = f1_score(labels, preds, average="macro", labels=[0,1], zero_division=0)

    return {"accuracy": acc, "f1_macro": f1}


In [4]:
def run_experiment(
    ds,
    task,
    train_variety=None,
    train_source=None,
    test_variety=None,
    test_source=None,
    model_name="roberta-large",
    seed=7,
    max_length=128,
    train_epochs=3,
    lr=2e-5,
    train_bs=16,
    eval_bs=32,
    fp16=True,
    output_dir_prefix="EXT"
):
    set_seed(seed)

    # 1) Filtra train e "test" (usiamo validation come test)
    train_full = filter_hf(ds["train"], task=task, variety=train_variety, source=train_source)
    test_set   = filter_hf(ds["validation"], task=task, variety=test_variety, source=test_source)

    # 2) Controlli minimi
    if len(train_full) < 50:
        return None, f"SKIP: train troppo piccolo ({len(train_full)})"
    if len(test_set) < 30:
        return None, f"SKIP: test troppo piccolo ({len(test_set)})"

    if not has_both_labels(train_full):
        return None, "SKIP: train ha una sola classe"

    warn_msg = None
    if not has_both_labels(test_set):
        warn_msg = "WARN: test ha una sola classe (F1 macro può essere poco informativa)"

    # 3) Split interno train/val (sklearn stratified)
    idx = np.arange(len(train_full))
    y = np.array(train_full["label"])

    # se y ha una sola classe, stratify fallisce (ma l'abbiamo già controllato)
    train_idx, val_idx = train_test_split(
        idx,
        test_size=0.1,
        random_state=seed,
        stratify=y
    )

    train_set = train_full.select(train_idx.tolist())
    val_set   = train_full.select(val_idx.tolist())

    # 4) Tokenizer + preprocess
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    def preprocess_function(batch):
        return tokenizer(
            batch["text"],
            truncation=True,
            max_length=max_length,
            padding="max_length"
        )

    train_set = train_set.map(preprocess_function, batched=True)
    val_set   = val_set.map(preprocess_function, batched=True)
    test_set  = test_set.map(preprocess_function, batched=True)

    cols = ["input_ids", "attention_mask", "label"]
    train_set.set_format("torch", columns=cols)
    val_set.set_format("torch", columns=cols)
    test_set.set_format("torch", columns=cols)

    # 5) Model
    model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=2
    )

    # 6) TrainingArguments
    outdir = (
        f"{output_dir_prefix}_{task}_"
        f"TR-{train_variety or 'ALL'}-{train_source or 'ALL'}_"
        f"TE-{test_variety or 'ALL'}-{test_source or 'ALL'}"
    )

    args = TrainingArguments(
        output_dir=outdir,
        num_train_epochs=train_epochs,
        learning_rate=lr,
        per_device_train_batch_size=train_bs,
        per_device_eval_batch_size=eval_bs,
        weight_decay=0.01,
        eval_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="f1_macro",
        logging_steps=100,
        fp16=fp16,
        report_to="none"
    )

    # 7) Trainer
    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=train_set,
        eval_dataset=val_set,
        compute_metrics=compute_metrics
    )

    trainer.train()

    # 8) Evaluate sul "test" (validation HF filtrata)
    test_metrics = trainer.evaluate(test_set)

    result = {
        "task": task,
        "train_variety": train_variety,
        "train_source": train_source,
        "test_variety": test_variety,
        "test_source": test_source,
        "n_train": len(train_full),
        "n_test": len(test_set),
        "test_accuracy": test_metrics.get("eval_accuracy"),
        "test_f1_macro": test_metrics.get("eval_f1_macro"),
        "note": warn_msg
    }
    return result, None


In [5]:

VARIETIES = ["en-AU", "en-IN", "en-UK"]

results = []
errors = []

TASK = "Sarcasm"
SOURCE_FIXED = "Reddit"

# ESEMPIO: fai solo una riga (train en-AU -> test en-IN/en-UK) per non farlo pesantissimo
TRAIN_VAR = "en-AU"
for test_var in VARIETIES:
    res, err = run_experiment(
        ds,
        task=TASK,
        train_variety=TRAIN_VAR,
        train_source=SOURCE_FIXED,
        test_variety=test_var,
        test_source=SOURCE_FIXED,
        model_name="roberta-large",
        output_dir_prefix="CROSSVAR"
    )
    if res is not None:
        results.append(res)
    else:
        errors.append({"setting": f"{TRAIN_VAR}->{test_var}", "error": err})

pd.DataFrame(results)


Filter:   0%|          | 0/17760 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2428 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

Map:   0%|          | 0/177 [00:00<?, ? examples/s]

Map:   0%|          | 0/230 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro
1,0.633000,0.675825,0.581921,0.367857


KeyboardInterrupt: 